In [3]:
import re
import json
import spacy
import deepl
import spacy
import pandas as pd
import numpy as np
import langdetect

In [4]:
import transformers
import torch

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
nlp = spacy.load('en_core_web_sm')

Structuring the text:

The following functions provide a way to get raw text and categorize it into the different figure numbers, the different sentences the subject spoke for each figure as well as storing information like the special position markers associated with the specific sentences they refer to.

In [6]:
raw_text = '''

I/ @ ^ Une sorte de coléoptère avec des grandes ailes
ou ça peut être une sorte de grenouille.
V Une sorte de masque Viking avec quatre yeux.
Une sorte de silhouette.
Un petit personnage avec  une bouche ouverte qui a l’air surpris
et son reflet là.


II/ V Deux personnages en train d’établir un contact ou
un combat parce qu’il y a du sang ou une jambe qui a été coupée.
Des poumons.
^ Une tête de coq avec la crête
Avion de chasse, planeur, feu des réacteurs ;
là aussi ça peut être un plus grand qui envoie des missiles
Des hommes troncs, peut-être irrationnels
ou des espèces de chenilles avec un gros ventre.

III/ ^ Deux guitares avec des formes bizarres, le manche
tordu.
Encore deux personnages face à face bizarrement constitués,
comme du Picasso. Ou deux personnages qui font qu’un.
Deux espèces de missiles ou de torpilles.
Des poumons
ou un papillon.
V Juste la partie d’un vieillard avec sa canne
ou un embryon.
Une espèce de grosse bête, on voit l’intérieur de son corps,
Les mandibules.

IV/ V Je vois encore une sorte de créature bizarre, un colosse,
Avec  les ailes là et des pattes étranges.
^ Deux personnages sous une espèce de burka.
V La tête de la mouche (film).
> Tête de bébé crocodile ou de cochon vu qu’il y a pas d’oreille
ou peut-être une taupe, une sorte de trompe.

V/ ^ Je vois aussi une sorte de mélange entre une chauve-souris
et un escargot, là les pattes sont un peu étranges.
@ Sinon là je vois rien d’autre.
Peut-être deux têtes de personnages en tout petit,
Ils sont dos à dos.
Le guitariste d’ACDC : sa tête avec ses cheveux.

VI/ @ ^ Une libellule un peu bizarre avec des ailes
à demi détruites.
Là, la mâchoire d’une bête avec des crochets à l’intérieur
de sa bouche, mais y a pas d’œil alors c’est peut-être
une plante carnivore.
L’homme sable (dans Spiderman) en train de reprendre
forme humaine.
Sinon c’est tout ce que je vois.
Ou encore des poumons qui sont cachés juste là
ou des sortes de graines de flageolets.
Ou là une sorte de personnage avec quelque chose
pour faire des bonds.
Finalement, c’est peut-être une sorte d’ange
extraterrestre qui va dans une île parce qu’il y a des ailes.
Il descend par là donc finalement c’est peut-être
un enfant qui va venir au monde.

VII/ V Deux espèces d’hippocampes avec une trompe ici ;
ils sont reliés les deux par quelque chose qui est là ;
ils ont l’air d’être congelés parce qu’il y a des stalactites,
comme peut-être des animaux préhistoriques qui ont été congelés.
^ Ou là sinon on peut voir des espèces de personnages
avec des bras en forme de tronçonneuse qui tentent de se rapprocher.
Des soldats de l’armée prussienne avec des pointes sur le casque.
Là on peut voir une espèce de tête avec un groin mais vu
qu’il y a une proéminence ça peut être le bouffon du roi.
Si on intègre le blanc on peut voir un masque.
V Une arme des Mayas ; ou peut-être pas une arme,
un objet Maya pour le culte
ou une espèce de champignon.

VIII/ V Une vue de microscope avec les colorants qu’il faut.
Deux espèces d’insectes qui sont en train de butiner ce qui est là.
Une tête de mouche avec quelque chose qui dégouline (orange).
Des cordes vocales (traits verts).
Un personnage avec des sortes d’antennes, en faisant
abstraction des cordes vocales.
Deux caméléons qui gravissent une montagne ;
on a l’impression qu’ils se serrent une main ;
il y a une transaction ou quelque chose comme ça :
ils donnent de l’argent ou se rechargent en énergie.

IX/ ^ Deux espèces de créatures difformes qui se rapprochent.
Deux narines, peut-être un crane qui est caché par tout le reste.
Une tête de fantôme en partie cachée par le reste
ou une sorte de champ magnétique.
Deux îles ou deux pays qui sont le reflet de l’autre (vert).
Un morceau de jambon de Parme (rose).
Des espèces de racines, comme le logo de Sépultura (traits oranges)
Une espèce de matière symbiotique qui fusionne.
Un personnage avec des ailes (vert).

X/ ^ Ca fait assez art contemporain, comme du Miro.
Deux espèces de petites bestioles qui se battent pour savoir
ce qui est au milieu (noir).
Des pieuvres ou gendarmes extraterrestres qui essayent
de les séparer (bleu).
Deux petits soleils ou comètes (jaune).
Deux chenilles ou la lettre Omega avec une araignée ou
diapason au boût (vert).
Deux sortes d’embryons qui recrachent une matière bleue (rose).
Deux espèces de fourmis avec une flamme (noir+jaune), elles sont
entrain de rattraper les deux choses qui sont là (orange)
ou la moto du Ghostrider.
V Je vois aussi deux yeux avec une bouche bizarre :
le vert c’est les cornes, le bleu la bouche, le gris un rasoir
électrique ou une sorte de tournevis sonique.
Deux personnages avec un ponpon bleu qui sont
entrain de se liquéfier, ils sont peut-être constitués de peau
et d’air (rose).
Les deux fourmis, elles vont essayer de mettre un chapeau
pour les reboucher pour pas qu’elles se dégonflent.
Deux brosses pour le dos ou à cheveux ou un compas
un peu bizarre (orange).
Finalement les deux chenilles il y a leurs projections astrales
qui est entrain de sortir de leur corps.
Deux personnes qui se soutiennent pour pas perdre l’équilibre
et pas tomber là (bleu central).

Choix + : X parce qu’on peut voir beaucoup de choses.
Choix - : II parce que ça a l’air de deux personnes qui sont entrain de se battre à mort, deux gladiateurs. Puis il y a aussi peut-être les espèces d’avions de guerre.


Rq : latences longues sauf à V et VIII
'''

Since the original text is in French but most of the resources are English, doing the translation after extracting the position markers may be more appropiate since the automatic translation may mess with those caracters, specially the V and mistranslate them. We are using the DeepL free API which allows for 500.000 characters a month.

Another note on DeepL: there are a lot of "figures" mentioned in the target text, that is, the translation. If you look at the prompt section, the llm seems to confuse it by figures as in art figures instead of figures as in human figures. I know that when doing the DeepL API call you can pass a glossary as an argument to the function. Maybe I would be a good idea to make a glossary with more concrete terms if there are synonyms to those semantically ambiguous nouns. Alternative: look for a way to measure the specificity of a word to deal with those kinds of nouns/terms

This part is commented out because there is no need to abuse the API limit, the ouput dictionary was saved as demo_text_preprocess so we can keep working on the different parts of the project.

As you can see, we are doing most of the parsing with regular expressions. The flow of the function is:
- 1st we split the text according to the roman numerals and we extract the Choix and Rq parts.
- 2nd we clean up the response text to avoid unnecesay extralines and we extract the special markers. At this point we have two full text variables: one raw that we will extract the positional markers from and a clean copy we'll send to DeepL.
- 3rd the clean text for each figure goes through the DeepL API and we store the translation for the whole text corresponding to the figure.
- 4th we split the text corresponding to each figure into its individual sentences.

- vikings regex take care that it doens ttake the initial V

# 1. Setting up the evaluate function using the classification models 

## FX for Bert Models

In [7]:
from transformers import BertModel, BertTokenizer
import torch
import numpy as np 
import shutil
from itertools import compress

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
classifications = ["individual_determinants", "macro_determinants", "individual_contents","macro_contents"]

def classification_info(classification):
    
    info_dictionary = {
        "individual_determinants":{"labels":['C', "C'", "C'F", 'CF', "CF'", 'CLOB', 'CLOBF', 
                                             'E', 'EF', 'F', 'FC', "FC'", 'FCLOB', 'FE', 'K', 
                                             'KAN', 'KOB', 'KP'],
                          "path":"best_model_RorschIA_individual_determinants.pt"},
        
        "macro_determinants":{"labels":['color_sum', 'threat_sum', 'fading_sum', 
                                        'form_sum', 'kinesthetics_sum'],
                          "path":"best_model_RorschIA_macro_determinants.pt"},
        
        "individual_contents":{"labels":['(A)', '(AD)', '(H)', '(HD)', 'A', 'ABS', 'AD', 
                                         'ALIM', 'ANAT', 'ARCH', 'ART', 'BOT', 'ELEM', 
                                         'FRAG', 'GÉO', 'H', 'HD', 'MQ', 'NAT', 'OBJ', 
                                         'PAYS', 'RADIO', 'SC', 'SCÈNE', 'SEX', 'SG', 'VÊT'],
                          "path":"best_model_RorschIA_individual_contents.pt"},
        
        "macro_contents":{"labels":['animal_sum', 'human_sum', 'abs_sum', 
                                    'food_sum', 'art_arch_sum', 'nature_sum', 
                                    'fragment_sum', 'geo_sum', 'object_sum', 
                                    'science_sum', 'graphic_sum'],
                          "path":"best_model_RorschIA_macro_contents.pt"}
        }
    labels = info_dictionary[classification]["labels"]
    path = info_dictionary[classification]["path"]
    
    return labels, path

def determine_model_class(labels, path):
    n_labels = len(labels)
    class BERTClass(torch.nn.Module):
        def __init__(self):
            super(BERTClass, self).__init__()
            # self.bert_model = LongformerModel.from_pretrained('allenai/longformer-base-4096', return_dict=True, problem_type="multi_label_classification")
            self.bert_model = BertModel.from_pretrained("bert-base-uncased", return_dict=True, problem_type="multi_label_classification")

            self.dropout = torch.nn.Dropout(0.3) 
            self.linear = torch.nn.Linear(768, n_labels) # have to change he n of possible labels here

        def forward(self, input_ids, attn_mask, token_type_ids):
            output = self.bert_model(
                input_ids,
                attention_mask=attn_mask,
                token_type_ids=token_type_ids
            )

            output_dropout = self.dropout(output.pooler_output)
            # print(output_dropout)
            output = self.linear(output_dropout)
            return output
        
    model = BERTClass()
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['state_dict'])
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    
    return model, device

def preprocess_text(text):

    encodings = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
)
    return encodings

def predict_prob(model, device, encodings):
    model.eval()

    sigmoid = torch.nn.Sigmoid()
    with torch.no_grad():
        input_ids = encodings['input_ids'].to(device, dtype=torch.long)
        attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)

        output = model(input_ids, attention_mask, token_type_ids)

        # prob_outputs = outputs.cpu().detach().numpy().tolist()
        probs = sigmoid(torch.Tensor(output)).cpu().detach().numpy().tolist()

    # final probabilities
    probs_unfolded = [prob for probs_list in probs for prob in probs_list]
    return probs_unfolded

def y_boolean(probabilities, classification):
    probabilities_array = np.array(probabilities)
    y_pred = np.zeros(probabilities_array.shape)
    y_pred[np.where(probabilities_array>=0.5)] = 1
    if "individual" in classification:
        class_type = classification.split("_")
        name_class = "canonical_" + class_type[1]
    else:
        name_class = classification
    if 1 not in y_pred:
        idx = np.argmax(probabilities_array)
        best_pred = probabilities_array[idx]
        y_pred[idx] = 1
        print("Low confidence prediction for {}".format(name_class))
    y_bool = y_pred.astype(bool)
    return y_bool

def bool_to_label(y_bool, labels):
    
    label_output = list(compress(labels, y_bool))
    return label_output

def evaluate_text(classification, text):
    
    labels, path = classification_info(classification)
    model, device = determine_model_class(labels, path)
    encodings = preprocess_text(text)
    probabilities = predict_prob(model, device, encodings)
    y_bool = y_boolean(probabilities, classification)
    label_output = bool_to_label(y_bool, labels)
    
    return label_output

c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
label = evaluate_text("individual_determinants", "a person with a lot of wings")
label

['F']

In [9]:
label = evaluate_text("macro_determinants", "a person with a lot of wings")
label

['form_sum']

In [10]:
label = evaluate_text("individual_contents", "a person with a lot of wings")
label

['HD']

In [11]:
label = evaluate_text("macro_contents", "a person with a lot of wings")
label

['human_sum']

# 2. Organizing the text into the corresponding figures and responses

In [14]:
def get_responses(raw_text):
  
  """This function takes plain text, parses the in-text markers to extract the text corresponding to each Rorschach figure,
  runs the responses through the Deepl API to translate them and returns a dictionary with the responses for each figure properly organized.
  """

  dict_numbers = {"I": 1, "II": 2,
                  "III": 3, "IV": 4,
                  "V": 5, "VI": 6,
                  "VII": 7, "VIII": 8,
                  "IX": 9, "X": 10}

  regex_response_number = '[A-Z]+/'                                # since the responses alway start with the number + \
  list_number_figure = re.findall(regex_response_number, raw_text)  # finds the regex for the number of responses
  text_i = re.split(regex_response_number, raw_text)

  # regex_choix_pos = r'Choix +(.*?)Choix -'

  # regex_choix_neg = r'Choix -(.*?)Rq :'

  # regex_rq = r'Rq :(.*?)\n'
  
  try: 
    text_choix_pos = re.findall(r'Choix \+ :(.*?)\nCh.?', raw_text)[0].strip()
    text_choix_neg = re.findall(r'Choix \- :(.*?)\n', raw_text)[0].strip()
    text_rq = re.findall(r'Rq :(.*?)\n', raw_text)[0].strip()
    additional_info = {"Choix_pos": text_choix_pos, "Choix_neg": text_choix_neg, "Rq": text_rq}
    
  except:
      additional_info = {"Choix_pos": "nada", "Choix_neg": "nada", "Rq": "nada"}
      
  list_responses = []
  for i in range(len(list_number_figure)):
    if list_number_figure[i] not in text_i[i]:
      j = i + 1 # there is a \n\n string at index 0 so the text actually starts at index 1
      # print("not found")
    else:
      j = i

    dict_responses = {}
    n_response = list_number_figure[i][:-1]
    number = dict_numbers[n_response]
    text_i = re.split(regex_response_number, raw_text)
    full_text = text_i[j].strip()

    regex_line_break = '\n++'

    list_sentences_raw = text_i[j].split(".")

    text_i[j] = re.sub(regex_line_break, " ", text_i[j]) \
                  .replace("@", "") \
                  .replace("^", "") \
                  .replace("V", "") \
                  .replace(">", "") \
                  .replace("  ", " ") \
                  .strip() \

    # DEEPL API TRANSLATION
    language = langdetect.detect(raw_text)
    
    if language == "fr":
      with open(r"DEEPL_KEY\DEEPL_API_KEY.txt", "r") as f:
        API_KEY = f.read()
      # print("API Key Found!")
      translator = deepl.Translator(API_KEY)
      result = translator.translate_text(text_i[j], target_lang="EN-US", preserve_formatting=True)
      text_i[j] = result.text

    dict_responses["figure_number"] = number
    dict_responses["raw_response"] = full_text
    dict_responses["clean_response"] = text_i[j]

    doc = nlp(text_i[j])
    list_sentences_clean = [sent for sent in doc.sents]
    clean_sentences = []

    special_markers_list = ["@", "^", "V ", "<", ">"]
    # structuring the sentences inside the while loop
    i = 0
    while i < len(list_sentences_clean):

      if "Choix" in list_sentences_clean[i]:  # if we reach choix, there are no more actual responses by the patient, those are just comments by the psychologist
        break
      elif "Choice" in list_sentences_clean[i]:  # if we reach choix, there are no more actual responses by the patient, those are just comments by the psychologist
        break

      j = i + 1 # for display purposes

      dict_sentence_info = {}
      dict_sentence_info["response"] = list_sentences_clean[i]
      dict_sentence_info["response_number"] = j
      clean_sentences.append(dict_sentence_info)

      list_special_markers = []
      for marker in special_markers_list:
        if marker in list_sentences_raw[i]:
          list_special_markers.append(marker)
          
      dict_sentence_info["special_marker"] = list_special_markers

      j = j + 1
      i = i + 1

    dict_responses["sentences"] = clean_sentences
    print("Figure", dict_responses["figure_number"])
    for dictionary in dict_responses["sentences"]:
      # print(dictionary)
      list_keys = list(dictionary.keys())
      # print(list_keys)
      for key in list_keys:
        if "response" == key:
          sentence = dictionary[key]
          print("\nResponse {}: ".format(dictionary["response_number"]), str(sentence))
          print("\nSpecial markers {}: ".format(key), dictionary["special_marker"])
          
          # applying the classification function here 
          
          for model in classifications:
              classification = evaluate_text(model, str(sentence))
              # print(classification)
              dictionary[model] = classification

          print("\nCanonical Contents:", dictionary["individual_contents"], "\nMacro Contents:", dictionary["macro_contents"])
          print("Canonical Determinants:", dictionary["individual_determinants"], "\nMacro Determinants:", dictionary["macro_determinants"], "\n")            

    list_responses.append(dict_responses)

  list_responses.append(additional_info)



  return list_responses

In [15]:
responses = get_responses(raw_text)
responses

Figure 1

Response 1:  Some kind of beetle with big wings, or it could be some kind of frog.

Special markers response:  ['@', '^']


c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Canonical Contents: ['A'] 
Macro Contents: ['animal_sum']
Canonical Determinants: ['F'] 
Macro Determinants: ['form_sum'] 


Response 2:  Some kind of iking mask with four eyes.

Special markers response:  ['V ']
Low confidence prediction for canonical_contents

Canonical Contents: ['MQ'] 
Macro Contents: ['object_sum']
Canonical Determinants: ['F'] 
Macro Determinants: ['form_sum'] 


Response 3:  Some kind of silhouette.

Special markers response:  []
Low confidence prediction for canonical_determinants
Low confidence prediction for canonical_contents
Low confidence prediction for macro_contents

Canonical Contents: ['HD'] 
Macro Contents: ['human_sum']
Canonical Determinants: ['F'] 
Macro Determinants: ['form_sum'] 


Response 4:  A small character with an open mouth who looks surprised and his reflection there.

Special markers response:  []
Low confidence prediction for canonical_determinants

Canonical Contents: ['HD'] 
Macro Contents: ['human_sum']
Canonical Determinants: ['K']

[{'figure_number': 1,
  'raw_response': '@ ^ Une sorte de coléoptère avec des grandes ailes\nou ça peut être une sorte de grenouille.\nV Une sorte de masque Viking avec quatre yeux.\nUne sorte de silhouette.\nUn petit personnage avec  une bouche ouverte qui a l’air surpris\net son reflet là.',
  'clean_response': 'Some kind of beetle with big wings, or it could be some kind of frog. Some kind of iking mask with four eyes. Some kind of silhouette. A small character with an open mouth who looks surprised and his reflection there.',
  'sentences': [{'response': Some kind of beetle with big wings, or it could be some kind of frog.,
    'response_number': 1,
    'special_marker': ['@', '^'],
    'individual_determinants': ['F'],
    'macro_determinants': ['form_sum'],
    'individual_contents': ['A'],
    'macro_contents': ['animal_sum']},
   {'response': Some kind of iking mask with four eyes.,
    'response_number': 2,
    'special_marker': ['V '],
    'individual_determinants': ['F'],
  

The proper clean single sentence responses in English are in the dictionary here: [{sentences: [{response_x}]}]

In [ ]:
demo = responses

In [123]:
# demo[0]

# 3. Function to turn the dictionary into a dataframe

In [170]:
# demo

In [20]:
evaluation_2 = demo.copy()

In [21]:
evaluation = evaluation_2.copy()

In [23]:
example = evaluation[0]["sentences"][0]

In [187]:
# demo

Function to clean up the non string values so that we don't have array length issues when saving the data to a frame

In [24]:
def clean_lists_evaluations(evaluation):
    list_keys = list(evaluation.keys())
    for key in list_keys:
        val = evaluation[key]
        if type(val) != str:
            clean = str(val).replace('\'', "").replace("[", "").replace("]", "")
            evaluation[key] = clean   
    return evaluation

In [25]:
clean_example = clean_lists_evaluations(example)

In [26]:
clean_example

{'response': 'Some kind of beetle with big wings, or it could be some kind of frog.',
 'response_number': '1',
 'special_marker': '@, ^',
 'individual_determinants': 'F',
 'macro_determinants': 'form_sum',
 'individual_contents': 'A',
 'macro_contents': 'animal_sum'}

In [27]:
df_2 = pd.DataFrame(clean_example, index=[0])
# df_2

In [28]:
df_2

,response,response_number,special_marker,individual_determinants,macro_determinants,individual_contents,macro_contents
0,"Some kind of beetle with big wings, or it coul...",1,"@, ^",F,form_sum,A,animal_sum


In [90]:
# final_df = pd.concat([df, df_2])
# final_df

### FX to put evey figure and the responses into the same df

In [29]:
def get_frame(list_dicts):
    """Turn the result dictionary into a DataFrame  
    """

    final_df = pd.DataFrame()

    for figure_dict in list_dicts:
        # print(figure_dict)
        # print(type(figure_dict))
        # print(figure_dict.keys())
        try:
            figure_n = figure_dict["figure_number"]
            list_sentences = figure_dict["sentences"]
        except:
            continue
        for sentence in list_sentences:
            # print(individual_response)
            individual_response = clean_lists_evaluations(sentence)
            individual_response["figure"] = figure_n
            # if we dont have consistent scalar values, it will crash
            response_data = pd.DataFrame(individual_response, index=[0])
            final_df = pd.concat([final_df, response_data])
    final_df = final_df.reset_index()
    
    return final_df

            

In [148]:
# evaluation

In [31]:
df = get_frame(evaluation)
df.head()

,index,response,response_number,special_marker,individual_determinants,macro_determinants,individual_contents,macro_contents,figure
0,0,"Some kind of beetle with big wings, or it coul...",1,"@, ^",F,form_sum,A,animal_sum,1
1,0,Some kind of iking mask with four eyes.,2,V,F,form_sum,MQ,object_sum,1
2,0,Some kind of silhouette.,3,,F,form_sum,HD,human_sum,1
3,0,A small character with an open mouth who looks...,4,,K,"color_sum, kinesthetics_sum",HD,human_sum,1
4,0,Two characters making contact or fighting beca...,1,V,KAN,kinesthetics_sum,SG,"human_sum, graphic_sum",2


# 4. Putting everything together

Finally, putting the functions together to generate a mini-report and the corresponding results dictionary

In [32]:
def raw_text_response_eval(raw_text):
    
    responses = get_responses(raw_text)
    
    results_dataframe = get_frame(responses)
    
    results_dataframe.to_csv("RorschIA_results.csv")
    
    return results_dataframe

In [33]:
evaluation = raw_text_response_eval(raw_text)

Figure 1

Response 1:  Some kind of beetle with big wings, or it could be some kind of frog.

Special markers response:  ['@', '^']


c:\Users\alber\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Canonical Contents: ['A'] 
Macro Contents: ['animal_sum']
Canonical Determinants: ['F'] 
Macro Determinants: ['form_sum'] 


Response 2:  Some kind of iking mask with four eyes.

Special markers response:  ['V ']
Low confidence prediction for canonical_contents

Canonical Contents: ['MQ'] 
Macro Contents: ['object_sum']
Canonical Determinants: ['F'] 
Macro Determinants: ['form_sum'] 


Response 3:  Some kind of silhouette.

Special markers response:  []
Low confidence prediction for canonical_determinants
Low confidence prediction for canonical_contents
Low confidence prediction for macro_contents

Canonical Contents: ['HD'] 
Macro Contents: ['human_sum']
Canonical Determinants: ['F'] 
Macro Determinants: ['form_sum'] 


Response 4:  A small character with an open mouth who looks surprised and his reflection there.

Special markers response:  []
Low confidence prediction for canonical_determinants

Canonical Contents: ['HD'] 
Macro Contents: ['human_sum']
Canonical Determinants: ['K']

12 minutes to run eveything

In [34]:
evaluation.head()

,index,response,response_number,special_marker,individual_determinants,macro_determinants,individual_contents,macro_contents,figure
0,0,"Some kind of beetle with big wings, or it coul...",1,"@, ^",F,form_sum,A,animal_sum,1
1,0,Some kind of iking mask with four eyes.,2,V,F,form_sum,MQ,object_sum,1
2,0,Some kind of silhouette.,3,,F,form_sum,HD,human_sum,1
3,0,A small character with an open mouth who looks...,4,,K,"color_sum, kinesthetics_sum",HD,human_sum,1
4,0,Two characters making contact or fighting beca...,1,V,KAN,kinesthetics_sum,SG,"human_sum, graphic_sum",2
